In [ ]:
import pandas
import json

f = open('user-wallet-transactions.json')
data = json.load(f)
f.close()
df = pandas.DataFrame(data)

In [ ]:
df['timestamp'] = pandas.to_datetime(df['timestamp'], unit='s')

def get_amount_from_action_data(x):
    try:
        d = json.loads(x)
        if 'amount' in d:
            return float(d['amount'])
        else:
            return 0
    except:
        return 0

df['amount'] = df['actionData'].apply(get_amount_from_action_data)

cols = ['amount', 'gasUsed', 'gasPrice', 'protocolFee']
for c in cols:
    if c in df.columns:
        df[c] = pandas.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(0)

In [ ]:
wallets_df = df.groupby('userWallet').count()
wallets_df['total_transactions'] = wallets_df['userWallet']
wallets_df['first_transaction_timestamp'] = df.groupby('userWallet')['timestamp'].min()
wallets_df['last_transaction_timestamp'] = df.groupby('userWallet')['timestamp'].max()
wallets_df = wallets_df.reset_index()

In [99]:
# Count how many things each wallet did
temp = df.groupby(['userWallet', 'action']).size()
temp = temp.unstack()
temp = temp.fillna(0)
wallets_df = wallets_df.set_index('userWallet')
wallets_df = wallets_df.join(temp)
wallets_df = wallets_df.reset_index()

# Check if 'amount' is in there
if 'amount' in df.columns:
    df['amount_numeric'] = pd.to_numeric(df['amount'], errors='coerce')
    df['amount_numeric'] = df['amount_numeric'].fillna(0)
else:
    df['amount_numeric'] = 0

# Get some totals
def get_dep(x):
    return x[df['action'] == 'deposit'].sum()

def get_borr(x):
    return x[df['action'] == 'borrow'].sum()

def get_rep(x):
    return x[df['action'] == 'repay'].sum()

def get_redm(x):
    return x[df['action'] == 'redeemUnderlying'].sum()

def get_liq(x):
    return x[df['action'] == 'liquidationCall'].sum()

total_amounts = df.groupby('userWallet').agg({
    'amount_numeric': [
        ('total_deposited', get_dep),
        ('total_borrowed', get_borr),
        ('total_repaid', get_rep),
        ('total_redeemed', get_redm),
        ('total_liquidation_calls_made', get_liq)
    ]
})

total_amounts.columns = total_amounts.columns.droplevel(0)
total_amounts = total_amounts.reset_index()

wallets_df = wallets_df.set_index('userWallet')
wallets_df = wallets_df.join(total_amounts.set_index('userWallet'))
wallets_df = wallets_df.reset_index()

# Calculate net and stuff
wallets_df['net_deposit'] = wallets_df['total_deposited'] - wallets_df['total_redeemed']
wallets_df['net_borrow_repay'] = wallets_df['total_repaid'] - wallets_df['total_borrowed']

# More calculations (hope it works)
wallets_df['wallet_age_days'] = (wallets_df['last_transaction_timestamp'] - wallets_df['first_transaction_timestamp']).dt.days
wallets_df['avg_transactions_per_day'] = wallets_df['total_transactions'] / (wallets_df['wallet_age_days'] + 1)

wallets_df['borrow_to_deposit_ratio'] = wallets_df['total_borrowed'] / (wallets_df['total_deposited'] + 0.000001)
wallets_df['repay_to_borrow_ratio'] = wallets_df['total_repaid'] / (wallets_df['total_borrowed'] + 0.000001)

wallets_df = wallets_df.replace([float('inf'), -float('inf')], 0)

ValueError: columns overlap but no suffix specified: Index(['borrow', 'deposit', 'liquidationcall', 'redeemunderlying', 'repay'], dtype='object')

In [84]:
wallets_df['credit_score_heuristic'] = 500
wallets_df['credit_score_heuristic'] += (wallets_df['net_deposit'] / 1000).clip(0, 100)
wallets_df['credit_score_heuristic'] += (wallets_df['repay_to_borrow_ratio'] * 50).clip(0, 200)
wallets_df['credit_score_heuristic'] += (wallets_df['total_transactions'] / 100).clip(0, 50)
wallets_df['credit_score_heuristic'] += (wallets_df['wallet_age_days'] / 30).clip(0, 50)
if 'liquidationCall_count' in wallets_df.columns:
    wallets_df['credit_score_heuristic'] -= (wallets_df['liquidationCall_count'] * 100).clip(0, 500)
wallets_df['credit_score_heuristic'] -= (wallets_df['borrow_to_deposit_ratio'] * 50).clip(0, 150)
wallets_df.loc[wallets_df['total_borrowed'] > 0, 'credit_score_heuristic'] -= (
    (1 - wallets_df['repay_to_borrow_ratio']) * 100
).clip(0, 100)
wallets_df['credit_score_heuristic'] = wallets_df['credit_score_heuristic'].clip(0, 1000).astype(int)


In [85]:
features = wallets_df.select_dtypes(include=np.number).columns.tolist()
features.remove('credit_score_heuristic')
X = wallets_df[features]
y = wallets_df['credit_score_heuristic']

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [87]:
scaler = StandardScaler()
X_train_scaled_df = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)


In [88]:
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train_scaled_df, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [89]:
y_pred = model.predict(X_test_scaled_df)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")


MAE: 0.13, RMSE: 1.59, R2: 0.58


In [90]:
joblib.dump(model, 'aave_credit_score_model.pkl')
joblib.dump(scaler, 'aave_credit_score_scaler.pkl')

['aave_credit_score_scaler.pkl']

In [103]:
def engineer_features_for_scoring(df_raw_transactions):
    print("Starting feature engineering for scoring...")

    df_raw_transactions['timestamp'] = pd.to_datetime(df_raw_transactions['timestamp'], unit='s')

    def get_amount_from_action_data(action_data_str):
        try:
            return float(json.loads(action_data_str).get('amount', 0))
        except Exception:
            return 0.0

    if 'actionData' in df_raw_transactions.columns:
        df_raw_transactions['amount'] = df_raw_transactions['actionData'].apply(get_amount_from_action_data)
    else:
        print("Warning: 'actionData' column not found. 'amount' column will be zeros.")
        df_raw_transactions['amount'] = 0.0

    for col in ['amount', 'gasUsed', 'gasPrice', 'protocolFee']:
        if col not in df_raw_transactions.columns:
            print(f"Warning: Column '{col}' not found in raw data. Filling with zeros.")
            df_raw_transactions[col] = 0.0
        df_raw_transactions[col] = pd.to_numeric(df_raw_transactions[col], errors='coerce').fillna(0)

    wallets_df = df_raw_transactions.groupby('userWallet').agg(
        total_transactions=('userWallet', 'size'),
        first_transaction_timestamp=('timestamp', 'min'),
        last_transaction_timestamp=('timestamp', 'max')
    ).reset_index()

    action_counts = df_raw_transactions.groupby(['userWallet', 'action']).size().unstack(fill_value=0)
    wallets_df = wallets_df.set_index('userWallet').join(action_counts, rsuffix='_count').reset_index()

    for action, name in [
        ('deposit', 'total_deposited'),
        ('borrow', 'total_borrowed'),
        ('repay', 'total_repaid'),
        ('redeemUnderlying', 'total_redeemed'),
        ('liquidationCall', 'total_liquidation_calls_made')
    ]:
        s = df_raw_transactions[df_raw_transactions['action'] == action].groupby('userWallet')['amount'].sum().rename(name)
        wallets_df = wallets_df.set_index('userWallet').join(s).fillna(0).reset_index()

    wallets_df['net_deposit'] = wallets_df['total_deposited'] - wallets_df['total_redeemed']
    wallets_df['net_borrow_repay'] = wallets_df['total_repaid'] - wallets_df['total_borrowed']
    wallets_df['wallet_age_days'] = (wallets_df['last_transaction_timestamp'] - wallets_df['first_transaction_timestamp']).dt.days
    wallets_df['avg_transactions_per_day'] = wallets_df['total_transactions'] / (wallets_df['wallet_age_days'] + 1)
    wallets_df['borrow_to_deposit_ratio'] = wallets_df['total_borrowed'] / (wallets_df['total_deposited'] + 1e-6)
    wallets_df['repay_to_borrow_ratio'] = wallets_df['total_repaid'] / (wallets_df['total_borrowed'] + 1e-6)
    wallets_df.replace([float('inf'), -float('inf')], 0, inplace=True)

    print("Feature engineering complete.")
    return wallets_df


In [104]:
def generate_wallet_scores_from_json(json_file_path, model_path='aave_credit_score_model.pkl', scaler_path='aave_credit_score_scaler.pkl'):
    with open(json_file_path, 'r') as f:
        df_raw = pd.DataFrame(json.load(f))
    features_df = engineer_features_for_scoring(df_raw)
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    model_features = model.feature_names_in_ if hasattr(model, 'feature_names_in_') else features_df.select_dtypes(include=np.number).columns.tolist()
    X_predict = features_df[model_features]
    credit_scores = np.clip(model.predict(scaler.transform(X_predict)), 0, 1000).astype(int)
    return pd.DataFrame({'userWallet': features_df['userWallet'], 'credit_score': credit_scores})

if __name__ == "__main__":
    input_json_file = 'user-wallet-transactions.json'
    if os.path.exists('aave_credit_score_model.pkl') and os.path.exists('aave_credit_score_scaler.pkl') and os.path.exists(input_json_file):
        final_scores_df = generate_wallet_scores_from_json(input_json_file)
        print(final_scores_df.head())
        final_scores_df.to_csv('wallet_credit_scores.csv', index=False)
        print("Wallet scores saved to wallet_credit_scores.csv")
    else:
        print("Required files not found. Please ensure 'aave_credit_score_model.pkl', 'aave_credit_score_scaler.pkl', and 'user-wallet-transactions.json' are in the correct directory.")



Starting feature engineering for scoring...
Feature engineering complete.
                                   userWallet  credit_score
0  0x00000000001accfa9cef68cf5371a23025b6d4b6           499
1  0x000000000051d07a4fb3bd10121a343d85818da6           499
2  0x000000000096026fb41fc39f9875d164bd82e2dc           500
3  0x0000000000e189dd664b9ab08a33c4839953852c           504
4  0x0000000002032370b971dabd36d72f3e5a7bf1ee           508
Wallet scores saved to wallet_credit_scores.csv
